In [ ]:
import numpy as np
import functools

def compute_C_and_D_star_functions(**params):
    """
    Create functions for computing C* and D* with memoization for the GARV model
    
    Parameters:
    -----------
    params: dict containing model parameters including numpy arrays
    
    Returns:
    --------
    Three functions for computing C* C_0* and D*
    """
    # Extract parameters from the params dictionary
    kappa = params['kappa']
    alpha_1 = params['alpha_1']
    beta_1 = params['beta_1']
    gamma_1_star = params['gamma_1_star']
    gamma_1 = params['gamma_1']
    lam = params['lam']
    omega_1 = params['omega_1']
    alpha_2 = params['alpha_2']
    beta_2 = params['beta_2']
    gamma_2_star = params['gamma_2_star']
    omega_2 = params['omega_2']
    sigma = params['sigma']
    rho = params['rho']
    r = params['r']

   
    # Create hashable versions of the numpy arrays
    params_hashable = {
        "kappa" : kappa,
        "alpha_1" : alpha_1,
        "beta_1" : beta_1,
        "gamma_1_star" : gamma_1_star,
        "lam" : lam,
        "omega_1" : omega_1,
        "alpha_2" : alpha_2,
        "beta_2" : beta_2, 
        "gamma_2_star" : gamma_2_star,
        "omega_2" : omega_2,
        "sigma" : sigma,
        "rho" : rho,
        "r": r
    }
    
    # Create a key for the memoization cache
    cache_key = hash(frozenset(params_hashable.items()))

    def compute_A_1(u, v_tuple):
        """
        Compute the A_1 function for GARV model
        
        Parameters:
        -----------
        u: complex number
        v_tuple: tuple, hashable version of v
        
        Returns:
        --------
        A_1 coefficient value
        """
        w_1 = v_tuple[0] * alpha_1
        w_2 = v_tuple[1] * alpha_2
        denom = 1 - 2 * w_2 * (1 - rho**2)
        a = w_1 + (w_2 * rho**2)/denom
        b = u - 2 * gamma_1_star * w_1 - (2 * rho * gamma_2_star * w_2)/denom
        c = gamma_1_star**2 * w_1 + (gamma_2_star**2 * w_2)/denom
                
        # Calculate A_1 using the parameters and v (w_R, w_RV)
        return kappa * (c + b**2/(2 * (1 - 2 * a)) + u * (lam - 0.5)) + v_tuple[0] * beta_1
    

    def compute_A_2(u, v_tuple):
        """
        Compute the A_2 function for GARV model
        
        Parameters:
        -----------
        u: complex number
        v_tuple: tuple, hashable version of v
        
        Returns:
        --------
        A_2 coefficient value
        """
        w_1 = v_tuple[0] * alpha_1
        w_2 = v_tuple[1] * alpha_2
        denom = 1 - 2 * w_2 * (1 - rho**2)
        a = w_1 + (w_2 * rho**2)/denom
        b = u - 2 * gamma_1_star * w_1 - (2 * rho * gamma_2_star * w_2)/denom
        c = gamma_1_star**2 * w_1 + (gamma_2_star**2 * w_2)/denom
        # Calculate A_2 using the parameters and v (w_R, w_RV)
        return (1 - kappa) * (c + ((b**2)/(2 * (1 - 2 * a))) + u * (lam - 0.5)) + v_tuple[1] * beta_2

    def compute_B(u, v_tuple):
        """
        Compute the B function for GARV model
        
        Parameters:
        -----------
        u: complex number
        v_tuple: tuple, hashable version of v
        
        Returns:
        --------
        B coefficient value
        """
        w_1 = v_tuple[0] * alpha_1
        w_2 = v_tuple[1] * alpha_2
        denom = 1 - 2 * w_2 * (1 - rho**2)
        a = w_1 + (w_2 * rho**2)/denom
        b = u - 2 * gamma_1_star * w_1 - (2 * rho * gamma_2_star * w_2)/denom
        c = gamma_1_star**2 * w_1 + (gamma_2_star**2 * w_2)/denom
        print("w_1", w_1)
        print("w_2", w_2)
        print("a", a)

        # Calculate B using the parameters and v (w_R, w_RV)
        term1 = -0.5 * np.log(1 - 2 * w_2 * (1 - rho**2))
        term2 = -0.5 * np.log(1 - 2 * a)
        term3 = u*r
        term4 = v_tuple[0] * omega_1
        term5 = v_tuple[1] * omega_2
        print(term1, term2, term3, term4, term5)
        return -0.5 * np.log(1 - 2 * w_2 * (1 - rho**2)) - 0.5 * np.log(1 - 2 * a) + u * r + v_tuple[0] * omega_1 + v_tuple[1] * omega_2


     # Define the memoized functions
    @functools.lru_cache(maxsize=None)
    def compute_C_star(u, v_tuple, t, T, key):
        """
        Compute the vectorized C function for GARV model
        Parameters:
        -----------
        u: complex number
        v_tuple: tuple, hashable version of v
        t, T: time periods
        key: hash to identify parameter set
        
        Returns:
        --------
        Tuple representation of B* vector
        """
    
        # Calculate components
        C_1 = compute_C1(u, v_tuple, t, T, key)
        C_2 = compute_C2(u, v_tuple, t, T, key)
        
        return (C_1, C_2)
    
    @functools.lru_cache(maxsize=None)
    def compute_C1(u, v_tuple, t, T, key):
        """
        Compute C1 coefficient from B-4 in Christoffersen
        
        Parameters:
        -----------
        u: complex number for the MGF parameter
        t, T: time periods
        key: hash to identify parameter set
        
        Returns:
        --------
        C1 coefficient value
        """
        # Terminal condition
        if T - t == 1:
            return compute_A_1(u, (0, 0))
        
        # Recursive calculation based on equation (14)
        # C1(u,t,T) corresponds to the coefficient of the short-run component s_t
        
        # First compute C1, C2 for the next time step
        C_star_tplus1 = compute_C_star(u, v_tuple, t+1, T, key)
        
        return compute_A_1(u, C_star_tplus1)
    
    @functools.lru_cache(maxsize=None)
    def compute_C2(u, v_tuple, t, T, key):
        """
        Compute C2 coefficient from equation (17) in Christoffersen
        
        Parameters:
        -----------
        u: complex number for the MGF parameter
        t, T: time periods
        key: hash to identify parameter set
        
        Returns:
        --------
        C2 coefficient value
        """
        # Terminal condition
        if T - t == 1:
            return compute_A_2(u, (0, 0))
        
        # Recursive calculation
        # C2(u,t,T) corresponds to the coefficient of the long-run component q_t
        
        # First compute C1, C2 for the next time step
        C_star_tplus1 = compute_C_star(u, v_tuple, t+1, T, key)

        # Get B_q component from B* function which is analogous to C2
        result = compute_A_2(u, C_star_tplus1)
        
        return result
    
    @functools.lru_cache(maxsize=None)
    def compute_D_star(u, v_tuple, t, T, key):
        """
        Compute D coefficient from equation (17) in Christoffersen
        
        Parameters:
        -----------
        u: complex number for the MGF parameter
        t, T: time periods
        key: hash to identify parameter set
        
        Returns:
        --------
        D coefficient value
        """
        # Terminal condition
        if T - t == 1:
            return compute_B(u, (0, 0))
        
        # Recursive calculation
        # D(u,t,T) corresponds to the constant term
        
        # First compute C1, C2, D for the next time step
        C_star_tplus1 = compute_C_star(u, v_tuple, t+1, T, key)
        D_star_tplus1 = compute_D_star(u, v_tuple, t+1, T, key)
    
        
        result = compute_B(u, C_star_tplus1) + D_star_tplus1
        
        return result
    
    def compute_C0_star(u, v_tuple, t, T, key):
        return 0
    
    def wrapped_compute_C_star(u, v, t, T):
        """
        Wrapper for C2 function
        """
        if isinstance(v, np.ndarray):
            v_tuple = tuple(map(complex, v))
        else:
            v_tuple = v if isinstance(v, tuple) else (v,)
        
        result = compute_C_star(u, v_tuple, t, T, cache_key)
        return np.array(result)
    
    def wrapped_compute_C0_star(u, v, t, T):
        """
        Wrapper for C2 function
        """
        if isinstance(v, np.ndarray):
            v_tuple = tuple(map(complex, v))
        else:
            v_tuple = v if isinstance(v, tuple) else (v,)
        
        result = compute_C0_star(u, v_tuple, t, T, cache_key)
        return np.array(result)
    
    def wrapped_compute_D_star(u, v, t, T):
        """
        Wrapper for D function
        """
        if isinstance(v, np.ndarray):
            v_tuple = tuple(map(complex, v))
        else:
            v_tuple = v if isinstance(v, tuple) else (v,)
        
        return compute_D_star(u, v_tuple, t, T, cache_key)
    
    return wrapped_compute_D_star, wrapped_compute_C_star, wrapped_compute_C0_star, compute_B

In [ ]:
# returns and options
kappa = 4.17e-2
lam = 1.08e1
alpha_1 = 2.2e-8
beta_1 = 9.83e-1
gamma_1_star = 8.7e2
gamma_1 = 8.59e2
omega_1 = 5.28e-15
alpha_2 = 1.65e-6
beta_2 = 2.89e-6
gamma_2_star = 7.73e2
omega_2 = 1.1e-9
sigma = 7.88e-6
rho = 4.02e-1

In [ ]:
# just returns
kappa = 3.95e-1
lam = 1.47
alpha_1 = 4.61e-6
beta_1 = 9.67e-7
gamma_1 = 4.57e2
omega_1 = 5.74e-12
alpha_2 = 2.57e-6
beta_2 = 4.07e-6
gamma_2 = 6.17e2
omega_2 = 5.84e-12
sigma = 7.50e-6
rho = 1.03e-1

In [ ]:
kappa = 3.78e-2
lam = 0
alpha_1 = 1.71e-8
beta_1 = 9.83e-1
gamma_1_star = 9.91e2
omega_1 = 5.95e-14
alpha_2 = 1.59e-6
beta_2 = 4.08e-6
gamma_2_star = 7.85e2
omega_2 = 3.47e-12
sigma = 1.04e-5
rho = 1.00

In [ ]:
gamma_1_star = gamma_1  + lam
gamma_2_star = gamma_2 - x

In [ ]:
from HedgeAndPrice import *

params = {
        "kappa" : kappa,
        "alpha_1" : alpha_1,
        "beta_1" : beta_1,
        "gamma_1_star" : gamma_1_star,
        "gamma_1" : gamma_1,
        "lam" : lam,
        "omega_1" : omega_1,
        "alpha_2" : alpha_2,
        "beta_2" : beta_2, 
        "gamma_2_star" : gamma_2_star,
        "omega_2" : omega_2,
        "sigma" : sigma,
        "rho" : rho,
        "r": 0
    }

# Create D*, C0*, and C* functions
D_star, C_star, C0_star, B = compute_C_and_D_star_functions(**params)

# Define option parameters
option_params = {
    'K': 100  # Strike price
}

# Current state
t = 0
T = 50
y_t =  np.log(100)# Log of price at time t
asset_unconditional_variance = 1.1676e-4
delta = 1
h_R = 1.11e-4
h_RV = 7.65e-5
# Initialize state vector with unconditional values
l_star_t = compute_l_t_star(h_R, h_RV, delta)
print(l_star_t)
r = params['r']  # Risk-free rate
R = 1.5  # Real part of the contour for European call option (R > 1)



In [ ]:
# Compute the risk-minimizing hedging position

xi_t_plus_1 = risk_minimizing_hedge(
    t, T, y_t, l_star_t, r,
    D_star, C0_star, C_star,
    f_check_call, option_params, R
)
print(f"Risk-minimizing hedging position: {xi_t_plus_1}")

In [ ]:

# Compute the option price
option_price_value = option_price(
    t, T, y_t, l_star_t, r,
    D_star, C0_star, C_star,
    f_check_call, option_params, R
)

print(f"Option price: {option_price_value}")

In [ ]:
def test_mgf_properties():
    """Test MGF properties to validate coefficient calculations"""
    # Initialize test parameters
    t = 0
    T_values = [1, 10, 30, 50]
    u_values = [0, 0.5, 1.0, 1.5]
    
    # Use your current l_star_t values
    # This should be the values you're currently using in your model
    
    print("\nMGF Property Tests:")
    print("===================")
    
    # Test 1: MGF(0) should be 1 (for any horizon)
    print("\nTest 1: MGF(0) should be 1")
    for T in T_values:
        mgf_val = np.exp(D_star(0, np.zeros(2), t, T) + 
                         y_t * C0_star(0, np.zeros(2), t, T) + 
                         np.dot(C_star(0, np.zeros(2), t, T), l_star_t))
        print(f"T={T}: MGF(0) = {mgf_val}")
    
    # Test 2: MGF(1) should be e^(r*T) (risk-neutral martingale property)
    print("\nTest 2: MGF(1) should be e^(r*T)")
    for T in T_values:
        mgf_val = np.exp(D_star(1, np.zeros(2), t, T) + 
                         y_t * C0_star(1, np.zeros(2), t, T) + 
                         np.dot(C_star(1, np.zeros(2), t, T), l_star_t))
        expected = np.exp(r * T)
        print(f"T={T}: MGF(1) = {mgf_val}, Expected = {expected}")
    
    # Test 5: Examine C_star coefficients directly
    print("\nTest 5: C_star coefficient examination")
    for u in [0, 1]:
        for T in T_values:
            c_star = C_star(u, np.zeros(2), t, T)
            print(f"u={u}, T={T}: C_star = {c_star}")

test_mgf_properties()

In [ ]:
# First, modify your risk_minimizing_hedge function slightly to handle the MGF values more carefully
def risk_minimizing_hedge_modified(t, T, y_t, l_star_t, r, D_star, C0_star, C_star, f_check, option_params, R):
    """Modified version with more careful handling of MGF calculations"""
    # For debugging, print key values to verify they're reasonable
    print(f"Initial values - y_t: {y_t}, l_star_t: {l_star_t}")
    
    # Test MGF calculations to verify they're reasonable
    test_z = complex(R, 0)
    test_psi_1 = np.exp(log_psi_1(t, test_z, y_t, l_star_t, D_star, C0_star, C_star, T))
    test_psi_2 = np.exp(log_psi_2(t, test_z, y_t, l_star_t, r, D_star, C0_star, C_star, T))
    print(f"Test MGF - psi_1: {test_psi_1}, psi_2: {test_psi_2}")
    
    # Compute integrand values directly at sample points
    sample_y_values = [-10, -1, 0, 1, 10]
    for y in sample_y_values:
        z = complex(R, y)
        log_psi_1_val = log_psi_1(t, z, y_t, l_star_t, D_star, C0_star, C_star, T)
        log_psi_2_val = log_psi_2(t, z, y_t, l_star_t, r, D_star, C0_star, C_star, T)
        psi_diff = np.exp(log_psi_2_val) - np.exp(log_psi_1_val)
        
        # Other components of the integrand
        exp_term = np.exp((z-1) * y_t)
        f_val = f_check(z, **option_params)
        
        # Full integrand value
        integrand_value = exp_term * psi_diff * f_val
        
        print(f"y={y}: log_psi_1={log_psi_1_val}, log_psi_2={log_psi_2_val}, diff={psi_diff}, integrand={integrand_value.real}")
    
    # Defining the integrand function with better numerical handling
    def integrand_for_quad(y):
        z = complex(R, y)
        
        # Calculate components with careful handling of large exponents
        log_psi_1_val = log_psi_1(t, z, y_t, l_star_t, D_star, C0_star, C_star, T)
        log_psi_2_val = log_psi_2(t, z, y_t, l_star_t, r, D_star, C0_star, C_star, T)
        
        # Use log-sum-exp trick for numerical stability
        if log_psi_2_val.real > log_psi_1_val.real:
            diff = np.exp(log_psi_2_val) * (1 - np.exp(log_psi_1_val - log_psi_2_val))
        else:
            diff = np.exp(log_psi_1_val) * (np.exp(log_psi_2_val - log_psi_1_val) - 1)
        
        exp_term = np.exp((z-1) * y_t)
        f_val = f_check(z, **option_params)
        
        # Calculate the integrand value
        integrand_value = exp_term * diff * f_val
        return integrand_value.real
    
    # Use adaptive integration with tighter tolerances
    total_integral, error = integrate.quad(
        integrand_for_quad, -50, 50, 
        limit=1000, epsabs=1e-10, epsrel=1e-10
    )
    
    # Calculate the log_psi_0_val more carefully
    log_psi_0_val = log_psi_0(t, y_t, l_star_t, r, D_star, C0_star, C_star)
    print(f"log_psi_0_val: {log_psi_0_val}, exp: {np.exp(log_psi_0_val)}")
    print("total integral", total_integral)
    
    # Computing the final result with careful handling of potential numerical issues
    denominator = 2 * (np.exp(log_psi_0_val) - 1) * np.pi
    if abs(denominator) < 1e-10:
        print("Warning: Very small denominator in hedge calculation")
        
    xi_t_plus_1 = np.exp(-r * (T - t)) * total_integral / denominator
    
    return xi_t_plus_1

In [ ]:
# Compute the risk-minimizing hedging position

xi_t_plus_1 = risk_minimizing_hedge_modified(
    t, T, y_t, l_star_t, r,
    D_star, C0_star, C_star,
    f_check_call, option_params, R
)
print(f"Risk-minimizing hedging position: {xi_t_plus_1}")

In [62]:
C_star(2, np.zeros(2),0, 10)

array([1.6474623 , 7.35205561])

In [63]:
D_star(2, np.zeros(2),0, 10)

np.float64(5.875494885900571e-05)